In [1]:
!pip install mip

In [23]:
from itertools import product
from mip import Model, xsum, minimize, BINARY

# Tiempo de tareas
#       0   1   2   3  Tareas
TT = [[65, 73, 63, 57],   # 0 Trabajadores
      [67, 70, 65, 58],   # 1
      [68, 72, 69, 55],   # 2
      [67, 75, 70, 59],   # 3
      [71, 69, 75, 57],   # 4
      [69, 71, 66, 59]]   # 5

m = Model()
x = [[m.add_var(f'x({i},{c})', var_type=BINARY) for c in range(4)]
     for i in range(6)]
z = m.add_var('z')

for i in range(6):
  for c in range(4):
    z += x[i][c]*TT[i][c]

for i in range(6):
    m += xsum(x[i][c] for c in range(4)) <= 1

for c in range(4):
    m += xsum(x[i][c] for i in range(6)) == 1

m.objective = z
m.optimize(max_nodes=30)

<OptimizationStatus.OPTIMAL: 0>

In [24]:
for i in range(6):
    print('[ ',end='')
    for c in range(4):
      print(x[i][c].x, end=' ')
    print(']')

[ 0.0 0.0 1.0 0.0 ]
[ 1.0 0.0 0.0 0.0 ]
[ 0.0 0.0 0.0 1.0 ]
[ 0.0 0.0 0.0 0.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 0.0 0.0 ]


In [25]:
m.objective.x

254.0

# Con Scipy

In [26]:
import numpy as np

In [46]:
TT = [[65, 73, 63, 57],   # 0 Trabajadores
      [67, 70, 65, 58],   # 1
      [68, 72, 69, 55],   # 2
      [67, 75, 70, 59],   # 3
      [71, 69, 75, 57],   # 4
      [69, 71, 66, 59]]
c = np.array(TT).reshape((24,))

In [41]:
A = np.zeros((10,24))
offset = 0
for f in range(6):
  A[f,offset:offset+4] = np.ones(4)
  offset += 4
for f in range(4):
  matriz = np.zeros((6,4))
  matriz[:,f] = np.ones(6)
  A[6+f,] = matriz.reshape(24,)

In [42]:
A

array([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 

In [43]:
bl = np.array([0,0,0,0,0,0,1,1,1,1])
bu = np.array([1,1,1,1,1,1,1,1,1,1])


In [44]:
from scipy.optimize import LinearConstraint
constraints = LinearConstraint(A, bl, bu)

In [47]:
from scipy.optimize import milp
res = milp(c=c, constraints=constraints)
res.x

array([-0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])

In [48]:
res.x.reshape((6,4))

array([[-0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])